# Introduction to Astropy

The [Astropy Project](https://www.astropy.org/index.html) is a community effort to develop a core package for astronomy using the Python programming language and improve usability, interoperability, and collaboration between astronomy Python packages. The **core astropy package** contains functionality aimed at professional astronomers and astrophysicists, but may be useful to anyone developing astronomy software. The Astropy Project also includes [**"affiliated packages"**](), Python packages that are not necessarily developed by the core development team, but share the goals of Astropy, and often build from the core package's code and infrastructure. 

Astropy has become the standard tool to work with astronomical data, since its creation in 2012, but specially since the first version was released in 2015. Its widespread acceptance is mainly due to the strong support of [NumFOCUS](https://numfocus.org/sponsored-projectshttps://numfocus.org/sponsored-projects) (sponsor of many scientific tools in Python) and the approval of several other important institutions, like the [STScI](https://www.stsci.edu/). Currently, Astropy core package has almost 30000 commits of 375 different contributors. 

The content of the current exercise has been adapted from this [Python course held at ICE](https://github.com/Python4AstronomersAndParticlePhysicists/PythonWorkshop-ICE/blob/master/notebooks/10_01_Astronomy_Astropy.ipynb) in 2017.

# Astropy Basics
In this section, we will explain the basics of what can be done with Astropy, such as working with internal units, opening FITS files, tables, spectra and WCS.

## Constants and Units
Astropy provides a large amount of astronomical constants... 

but warning: The use of units can slow down the processing of a large data set.

In [ ]:
import numpy as np
from astropy import constants as const
from astropy import units as u

By default astropy constants uses S.I. units...

In [ ]:
print(const.c)

It can be transformed to any units...

In [ ]:
const.c.to('km/s')

In [ ]:
const.c.to('pc/yr')

You can also define your own constant using astropy Units

In [ ]:
my_emission_line_flux = 12.32 * u.erg / u.cm ** 2 / u.s
my_emission_line_flux

Here we can compute Earth's orbit speed using Astropy constants and asuming a circular orbit:

$$v=\frac{2\cdot\pi\cdot R}{P}$$

where $v$ is the speed of the Earth, $R$ is the Earth orbital radius and $P$ is the Earth orbital period.

In [ ]:
speed_of_earth = const.au * 2 * np.pi / u.yr
speed_of_earth.to('km/s')

Or it can also be computed (approximately) from the gravitational force between the Earth and the Sun:

$$v=\sqrt{\frac{G\cdot M_\odot}{R}}$$

where $M_\odot$ is now the mass of the Sun and $G$ is the gravitational constant.

In [ ]:
speed_of_earth_2 = np.sqrt(const.G * const.M_sun / const.au) 
speed_of_earth_2.to('km/s')

It can also be used to compute the orbital period of a satellite around the Earth, from its semi-major axis:

$$P=\frac{2\cdot\pi}{\sqrt{G\cdot M_\oplus}}\cdot a^{\frac{3}{2}}$$

In [ ]:
period = 2 * np.pi * (36000 * u.km + const.R_earth)**1.5 / np.sqrt(const.G * const.M_earth)
print(period.to(u.day))

### Celestial Coordinates

The simplest coordinate we can define is a single point in the sky, by default in the ICRS frame.

In [ ]:
from astropy.coordinates import SkyCoord
c = SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, frame='icrs')

#### Definition
It can be defined in almost any format used in astronomy (and there are many, as usual...) all representing the same location.

In [ ]:
c = SkyCoord(10.625, 41.2, frame='icrs', unit='deg')
c = SkyCoord('00h42m30s', '+41d12m00s', frame='icrs')
c = SkyCoord('00h42.5m', '+41d12m')
c = SkyCoord('00 42 30 +41 12 00', unit=(u.hourangle, u.deg))
c = SkyCoord('00:42.5 +41:12', unit=(u.hourangle, u.deg))
c

Astropy also has a significantly large list of sources than can be retrieved by its name:

In [ ]:
a_big_blue_star = SkyCoord.from_name('rigel')
print (a_big_blue_star.ra, a_big_blue_star.dec, a_big_blue_star.distance)

#### Transformation
We can easily convert to other coordinate systems, like the galactic...

In [ ]:
c.galactic

Or even get what is the closest constellation to the object, very useful for astronomers as you know...

In [ ]:
print(c.to_string('hmsdms'), 'is in', c.get_constellation(), 'and Rigel is in', a_big_blue_star.get_constellation())


#### Distances
Coordinates allow also to define distances:

In [ ]:
c = SkyCoord(ra=10.68458*u.degree, dec=41.26917*u.degree, distance=770*u.kpc)
print (c.cartesian.x, c.cartesian.y, c.cartesian.z)

If we define one or more coordinates we can compute the distance between the two objects:

In [ ]:
c1 = SkyCoord(ra=10*u.degree, dec=9*u.degree, distance=10*u.pc, frame='icrs')
c2 = SkyCoord(ra=11*u.degree, dec=10*u.degree, distance=11.5*u.pc, frame='icrs')

print ("Angular Separation: %s" % c1.separation(c2))
print ("Distance between objects: %s" % c1.separation_3d(c2))

#### Catalogue of sources

A catalogue of positions can also be created using numpy arrays:

In [ ]:
ras = np.array([0-.7, 21.5, 120.9]) * u.deg  
decs = np.array([4.5, -5.2, 6.3]) * u.deg   
catalogue = SkyCoord(ras, decs, frame='icrs')
catalogue.galactic

## Time and Date
The astropy.time package provides functionality for manipulating times and dates used in astronomy, such as UTC or MJD.
#### Definition

In [ ]:
from astropy.time import Time
times = ['2017-09-13T00:00:00', '2017-09-15T11:20:15.123456789',]
t1 = Time(times)
t1

Default format is ISOT and scale UTC, but it can be set to others.

In [ ]:
times = [58009, 58011.47239726]
t2 = Time(times, format='mjd', scale='tai')
t2

#### Format conversion

In [ ]:
print ("To julian date: %s" % t1[0].jd)
print ("To modified julian date: %s" % t1[0].mjd)
print ("To FITS: %s" % t1[0].fits)
print ("To GPS: %s" % t1[0].gps)
print ("To Bessel Epoch Year: %s" % t1[0].byear_str)
print ("To Julian Epoch Year: %s" % t1[0].jyear_str)

#### Timezones

When a Time object is constructed from a timezone-aware datetime, no timezone information is saved in the Time object. However, Time objects can be converted to timezone-aware datetime objects:

In [ ]:
from datetime import datetime
from astropy.time import Time, TimezoneInfo
import astropy.units as u
utc_plus_one_hour = TimezoneInfo(utc_offset=1*u.hour)
dt_aware = datetime(2000, 1, 1, 0, 0, 0, tzinfo=utc_plus_one_hour)
t = Time(dt_aware)  # Loses timezone info, converts to UTC
print(t)            # will return UTC

In [ ]:
print(t.to_datetime(timezone=utc_plus_one_hour)) # to timezone-aware datetime

## Additional tools

Astropy is an ecosystem by itself. Knowing well its possibilities will enable fast data analyses without having to worry too much on the nuts and bolts of them. Astropy has arrived to stay, for some time, at least. 